In [2]:
import numpy as np
import pandas as pd
import matplotlib
print(matplotlib.__version__)
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.lines as mlines
from glob import glob
from collections import Counter
import ns
import history_cmc as hic
import math
import scipy
from scipy.interpolate import interp1d
from scipy import stats
import matplotlib.cm as cm
import matplotlib as mpl
import random
from random import shuffle
import gzip
import sys
import astropy
from astropy import units

import gw_ecc_calc as gwcalc
import unit_convert as uc
import merger_rate_calculator as mr
import ns_tidalcapture as tc
import conversions
import dynamics as dyn


%matplotlib inline

matplotlib.rcParams.update({'font.size': 24})

sys.path.insert(1, '/fs/lustre/cita/claireshiye/cmctoolkit')
import cmctoolkit as cmct
      
twopi=2.*np.pi
yearsc=3.1557*10**7
Kconst=9.87*10**-48 ##yr/G^2
Gconst=6.674*10**-8 ##cm3*g-1*s-2
Gconst_sun = 4.30091*10**-3 ##pc*M_sun**-1*(km/s)^2
clight=3*10**10 ##cm/s
Msun=2*10**33 ##gram
AU=1.496*10**13  ##cm
AU_Rsun=214.93946938362 ##AU to R_sun
PC=3.086*10**18  ##cm
PC_Rsun = 44334448.0068964 ##pc to R_sun

3.5.2


In [3]:
path = '/fs/lustre/cita/claireshiye/CMC-COSMIC/CMC/ngc1851/N1.2e6rv0.5z0.0013rg2/'
snap_h5 = 'initial.snapshots.h5'
snap = cmct.Snapshot(fname=path+snap_h5, snapshot_name='/740(t=0.5006529)', conv=path+'initial.conv.sh', 
                     dist=4.52, # distance to cluster in kpc
                     z=0.0013)

##First Check

##Surface brightness and Velocity dispersion profile
E_BV = 0.04
d47 = 4.52  ##kpc
Av = 3.1*E_BV


###############################################
##Make surface brightness profile
snap.add_photometry('/fs/lustre/cita/claireshiye/cmctoolkit/filt_index.txt')
v_bincenter, v_profile = snap.make_smoothed_brightness_profile('V', bins=80,
                                                               min_mass=None, max_mass=None,
                                                               max_lum=12, fluxdict=None,
                                                               startypes=np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                                               min_logr=-2.0)


# Make velocity dispersion profiles
star_velbin_center, star_veldisp_profile, star_e_veldisp_profile = snap.make_smoothed_veldisp_profile(bins=80,
                                                                 min_mass=0.85,
                                                                 max_mass=None,
                                                                 dmax=None,
                                                                 fluxdict=None,
                                                                 startypes=np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                                                 min_logr=-2.0)

star_velbin_arcsec = conversions.pc_to_arcsec(star_velbin_center,d47)


##Make number density profile
star_numbin_center, star_profile, star_e_profile = snap.make_smoothed_number_profile(bins=80,
                                                 min_mass=0.85,
                                                 max_mass=None,
                                                 fluxdict=None,
                                                 startypes=np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                                 min_logr=-2.0)

star_numbin_arcsec = conversions.pc_to_arcsec(star_numbin_center,d47)
star_profile_arcsec = star_profile/(conversions.pc_to_arcsec(1.,d47)**2)
star_e_profile_arcsec = star_e_profile/(conversions.pc_to_arcsec(1.,d47)**2)



##Plotting
fig, ax = plt.subplots(nrows = 3, figsize = (15, 20), sharex = True)
ax[0].scatter(10**arcsec_t, SB_t,facecolor='gold',edgecolor='black',s=40,label=r'$\rm{Trager\,et\,al.\,1995}$')
ax[0].plot(v_bincenter, v_profile, c='g', label='V')
#ax[0].legend(loc='lower left')
#ax[0].set_xlabel('$r$ (arcsec)')
ax[0].set_ylabel(r'$\Sigma (mag/arcsec^{-2})$')
ax[0].set_xscale('log')
ax[0].set_xlim(0.3, 3e3)
ax[0].set_ylim(28, 10)


ax[1].scatter(r_bg_rv, sigma_bg_rv, s=30, color = 'orange',edgecolor='black')
ax[1].errorbar(r_bg_rv, sigma_bg_rv, yerr=[del_sigmau_bg_rv, del_sigmad_bg_rv], fmt='o', lw = 2.0, color = 'orange')
ax[1].scatter(r_ka, sigma_ka, s=30, color = 'gold', edgecolor='black')
ax[1].errorbar(r_ka, sigma_ka, yerr=[sigma_ka_plus, sigma_ka_minus], fmt='o', lw=2.0, color = 'gold')
ax[1].plot(star_velbin_arcsec, star_veldisp_profile, c='r', label='Stars')
ax[1].fill_between(star_velbin_arcsec, star_veldisp_profile - star_e_veldisp_profile,
                                  star_veldisp_profile + star_e_veldisp_profile, color='r', alpha=0.3)
#ax[1].legend(loc='lower left')
#ax[1].set_xlabel('$r$ (arcsec)')
ax[1].set_ylabel('$\sigma_v$ (km s$^{-1}$)')
ax[1].set_xscale('log')


ax[2].errorbar(arcsec_sd_deboer, sd_obs_deboer, yerr = sd_err_deboer,
                      fmt='o', c='gold', lw=2.0, label=r'$\rm{De\ Boer\,et\,al.\,2019}$')
ax[2].errorbar(arcsec_sd_miocchi, sd_obs_miocchi, yerr = sd_err_miocchi, fmt='o',
                      c='green', lw=2.0, label=r'$\rm{Miocchi\,et\,al.\,2013}$')
ax[2].plot(star_numbin_arcsec, star_profile_arcsec, c='r', label='Stars')
ax[2].fill_between(star_numbin_arcsec, star_profile_arcsec - star_e_profile_arcsec,
                                  star_profile_arcsec + star_e_profile_arcsec, color='r', alpha=0.3)

#ax[2].legend(loc='lower left')
ax[2].set_xlabel('$r$ (pc)')
ax[2].set_ylabel('$\Sigma_N$ (stars arcsec$^{-2}$)')
ax[2].set_xscale('log')
ax[2].set_yscale('log')

fig.tight_layout()

FileNotFoundError: File /projects/b1095/syr904/cmc/CMC-COSMIC/master_tc_test/ver_0601/MOCHA47Tuc_elson_rv4_3e6_tcon/initial.snapshots.h5 does not exist